In [1]:
%pylab
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid_at_node
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np
from matplotlib import colors,  cm
import matplotlib.animation as animation

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
mg = RasterModelGrid((5,10),500)
junc_elev = mg.add_zeros('node', 'junction__elevation')
R = mg.add_zeros('node', 'input__discharge')
#d_h = mg.add_ones('link', 'hydraulic__diameter')
thickness = 500.*np.ones(mg.number_of_nodes)
Z = mg.add_field('node', 'ice__thickness', thickness)
#a = mg.add_ones('link', 'head_loss__exponent')
#f = mg.add_ones('link', 'friction__factor')
#h = np.random.rand(mg.number_of_nodes)
h = mg.add_zeros('node', 'hydraulic__head')
Q = mg.add_zeros('link', 'conduit__discharge')
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)
#set boundary node head

#add input to node 310
moulin_Q=1.
mg.at_node['input__discharge'][12]=moulin_Q
#set heads at edges
h[mg.nodes_at_left_edge] = 400.
h[mg.nodes_at_right_edge] = 300.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
mg.set_closed_boundaries_at_grid_edges(False,True,True,True)
Q[mg.active_links] = 0.1*np.random.rand(mg.number_of_active_links)
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print("Number of links = ", mg.number_of_links)
print("Number of nodes = ", mg.number_of_nodes)
print("Number of active links = ", mg.number_of_active_links)
print("Number of core nodes = ", mg.number_of_core_nodes)

d_h = mg.add_zeros('link','hydraulic__diameter')
mg.at_link['hydraulic__diameter'][mg.active_links]= 0.5*np.random.rand(mg.number_of_active_links)




#code added for fsolve algorithm
#dhdx = mg.add_zeros('link', 'hydraulic_head_gradient')
#net_node_flux = mg.add_ones('node', 'net_node_flux')


pfn = PresFlowNetwork(mg)
pfn.run_one_step()
mc = MeltCreep(mg, dt=250.)

Number of links =  85
Number of nodes =  50
Number of active links =  40
Number of core nodes =  24
Number of iterations = 1 tolerance = 0.96378851429  iteration time= 0.00874185562133789
Number of iterations = 2 tolerance = 0.435654192598  iteration time= 0.008198738098144531
Number of iterations = 3 tolerance = 0.119390020377  iteration time= 0.009371280670166016
Number of iterations = 4 tolerance = 0.0134736012851  iteration time= 0.009763956069946289
Number of iterations = 5 tolerance = 0.00367091391148  iteration time= 0.00806117057800293
Number of iterations = 6 tolerance = 0.00312297288839  iteration time= 0.00778508186340332
Number of iterations = 7 tolerance = 0.00448133528767  iteration time= 0.008232593536376953
Number of iterations = 8 tolerance = 0.0126287879545  iteration time= 0.007361888885498047
Number of iterations = 9 tolerance = 0.00629358853313  iteration time= 0.007565736770629883
Number of iterations = 10 tolerance = 0.0031056594446  iteration time= 0.00944733619

/home/mcoving/Data/github/landlab/landlab/components/conduit_networks/pressurized_flow.py:155: RuntimeWarning: divide by zero encountered in true_divide
  self.r = 8*f*L/(g*r_s**2.*d_h**5.)


In [14]:
#Version of function that uses matplotlib to create plots rather than PIL
#def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='viridis', magnitude= False, lw=5,x_name='X', y_name='Y', var_name=''):#, logvalues=False ):
#    link_head_x = grid.node_x[grid.node_at_link_head]
#    link_head_y = grid.node_y[grid.node_at_link_head]
#    link_tail_x = grid.node_x[grid.node_at_link_tail]
#    link_tail_y = grid.node_y[grid.node_at_link_tail]       
#    if magnitude:
#        values = abs(grid.at_link[value_name])
#    else:
#        values = grid.at_link[value_name]
#    #if logvalues:
#    #    values = np.log10(abs(grid.at_link[value_name]))
#        
#    #Normalize color values
#    if autoscale:
#        cnorm = colors.Normalize()
#        cnorm.autoscale(values)
#    else:
#        cnorm = colors.Normalize(vmin,vmax)
#    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
#    scalarMap.set_array(values)
#    #set_cmap()
#    for i, value in enumerate(values):
#    #    draw.line( ((link_head_x[i]*pixel_xscaling_factor,link_head_y[i]*pixel_yscaling_factor),(link_tail_x[i]*pixel_xscaling_factor,link_tail_y[i]*pixel_yscaling_factor)),fill=scalarMap.to_rgba(value,bytes=True),width=lw) 
#        xs = [link_head_x[i],link_tail_x[i]]
#        ys = [link_head_y[i],link_tail_y[i]]
#        img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))
#    #imshow(np.asarray(im), origin='lower', extent=(0,grid.number_of_cell_columns,0,grid.number_of_cell_rows))
#    cb = colorbar(scalarMap)
#    cb.ax.set_ylabel(var_name)
#    xlabel(x_name)
#    ylabel(y_name)

In [4]:
from PIL import Image
from PIL import ImageDraw

def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='viridis', magnitude= False, lw=5, pixels_per_node=15, x_name='X', y_name='Y', var_name='', use_PIL=True):#, logvalues=False ):
    if use_PIL:
        pixel_xscaling_factor = pixels_per_node/grid.dx
        pixel_yscaling_factor = pixels_per_node/grid.dy
        im = Image.new('RGBA', ( int(round(grid.number_of_cell_columns*pixel_xscaling_factor)), int(round(grid.number_of_cell_rows*pixel_yscaling_factor)) ), (255,255,255,255))
        draw = ImageDraw.Draw(im)

    link_head_x = grid.node_x[grid.node_at_link_head]
    link_head_y = grid.node_y[grid.node_at_link_head]
    link_tail_x = grid.node_x[grid.node_at_link_tail]
    link_tail_y = grid.node_y[grid.node_at_link_tail]       
    if magnitude:
        values = abs(grid.at_link[value_name])
    else:
        values = grid.at_link[value_name]
    #if logvalues:
    #    values = np.log10(abs(grid.at_link[value_name]))
    
    
    #Normalize color values
    if autoscale:
        cnorm = colors.Normalize()
        cnorm.autoscale(values)
    else:
        cnorm = colors.Normalize(vmin,vmax)
    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
    scalarMap.set_array(values)
    #set_cmap()
    if use_PIL:
        for i, value in enumerate(values):
            draw.line( ((link_head_x[i]*pixel_xscaling_factor,link_head_y[i]*pixel_yscaling_factor),(link_tail_x[i]*pixel_xscaling_factor,link_tail_y[i]*pixel_yscaling_factor)),fill=scalarMap.to_rgba(value,bytes=True),width=lw) 
        imshow(np.asarray(im), origin='lower', extent=(0,grid.number_of_cell_columns,0,grid.number_of_cell_rows))
    else:
        for i, value in enumerate(values):
            xs = [link_head_x[i],link_tail_x[i]]
            ys = [link_head_y[i],link_tail_y[i]]
            img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))    
    cb = colorbar(scalarMap)
    cb.ax.set_ylabel(var_name)
    xlabel(x_name)
    ylabel(y_name)


In [6]:
nsteps = 10000
step_start = 1
fig1, axs = subplots(2,2,figsize=(10,6))
every = 50
for step in arange(nsteps)+step_start:
    print("step =",step, " avg d_h=",mg.at_link['hydraulic__diameter'].mean())
    pfn.run_one_step()    
    #mg.calc_grad_at_link(h, out=dhdx)
    #h = fsolve(network_residuals, h[mg.core_nodes])
    #Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    mc.run_one_step()
    if (step % every)==0: #make an animation frame
        subplot(2,2,1)
        imshow_grid_at_node(mg, h, var_name='Head ($m$)', cmap='viridis',vmin=300,vmax=400)
        #plot(mg.at_node['hydraulic__head'][mg.core_nodes])
        #ylim([300,400])
        subplot(2,2,2)
        plot_links(mg, 'conduit__discharge', magnitude=True, var_name='Discharge ($m^3/s$)', vmin=0., vmax=1, autoscale=False, use_PIL=False)
        #plot(mg.at_link['conduit__discharge'][mg.active_links])
        #ylabel('')
        #ylim([0,1.5])
        subplot(2,2,3)
        plot_links(mg, 'hydraulic__diameter', var_name='Hydraulic diameter ($m$)', autoscale=False, vmin=0, vmax=1.25, use_PIL=False)
        subplot(2,2,4)#, position=[0.2,0.2,0.9,0.9])
        plot(mg.at_link['hydraulic__diameter'][mg.active_links])
        xlabel('Link number')
        ylabel('Hydraulic diameter ($m$)')
        ylim(0,1.25)
        #ylim([0.5,1.5])
        image_name = 'heads_and_discharge'+str(step).zfill(6)+'.png'
        #tight_layout()
        subplots_adjust(wspace=0.3)
        savefig(image_name)
        fig1.clf()

#im_ani = animation.ArtistAnimation(fig1,ims)

step = 1  avg d_h= inf
Number of iterations = 1 tolerance = nan  iteration time= 0.009244203567504883
max ddh =  nan
step = 2  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.008061408996582031
max ddh =  nan
step = 3  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.00762176513671875
max ddh =  nan
step = 4  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.008041858673095703
max ddh =  nan
step = 5  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.00743412971496582
max ddh =  nan
step = 6  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007092952728271484
max ddh =  nan
step = 7  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.006931304931640625
max ddh =  nan
step = 8  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007380008697509766
max ddh =  nan
step = 9  avg d_h= nan
Number of iterations = 1 tolerance = nan  i

/home/mcoving/Data/github/landlab/landlab/components/conduit_networks/pressurized_flow.py:213: RuntimeWarning: invalid value encountered in double_scalars
  ADA_ii += 1./(node_as[link_idx]*node_rs[link_idx]*np.fabs(link_Q)**(node_as[link_idx]-1.))
/home/mcoving/Data/github/landlab/landlab/components/conduit_networks/pressurized_flow.py:227: RuntimeWarning: invalid value encountered in double_scalars
  ADA_ij[i][j] = -1./(a[this_link]*r[this_link]*np.fabs(self.Q[this_link])**(a[this_link]-1))
/home/mcoving/anaconda3/envs/landlab_dev/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:198: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)
/home/mcoving/Data/github/landlab/landlab/components/conduit_networks/pressurized_flow.py:237: RuntimeWarning: invalid value encountered in multiply
  dQ= -(1./a[self.grid.active_links])*self.Q[self.grid.active_links] - 1./(a[self.grid.active_links]*r[self.grid.active_links]*np.fabs(self

Number of iterations = 1 tolerance = nan  iteration time= 0.00822305679321289
max ddh =  nan
step = 26  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.00762486457824707
max ddh =  nan
step = 27  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.008929729461669922
max ddh =  nan
step = 28  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.009720087051391602
max ddh =  nan
step = 29  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007946968078613281
max ddh =  nan
step = 30  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007185935974121094
max ddh =  nan
step = 31  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.0075414180755615234
max ddh =  nan
step = 32  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007315158843994141
max ddh =  nan
step = 33  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time=

Number of iterations = 1 tolerance = nan  iteration time= 0.010178327560424805
max ddh =  nan
step = 97  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007534503936767578
max ddh =  nan
step = 98  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.00768280029296875
max ddh =  nan
step = 99  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.009794235229492188
max ddh =  nan
step = 100  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007779121398925781
max ddh =  nan
step = 101  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007384777069091797
max ddh =  nan
step = 102  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007379770278930664
max ddh =  nan
step = 103  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.0076389312744140625
max ddh =  nan
step = 104  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration

Number of iterations = 1 tolerance = nan  iteration time= 0.007843017578125
max ddh =  nan
step = 177  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007402896881103516
max ddh =  nan
step = 178  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.011258363723754883
max ddh =  nan
step = 179  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.00800180435180664
max ddh =  nan
step = 180  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007293224334716797
max ddh =  nan
step = 181  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007121562957763672
max ddh =  nan
step = 182  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007467746734619141
max ddh =  nan
step = 183  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.0070459842681884766
max ddh =  nan
step = 184  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration

Number of iterations = 1 tolerance = nan  iteration time= 0.008139848709106445
max ddh =  nan
step = 249  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.009155035018920898
max ddh =  nan
step = 250  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007218599319458008
max ddh =  nan
step = 251  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.008308649063110352
max ddh =  nan
step = 252  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007008552551269531
max ddh =  nan
step = 253  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.007466554641723633
max ddh =  nan
step = 254  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.00714874267578125
max ddh =  nan
step = 255  avg d_h= nan
Number of iterations = 1 tolerance = nan  iteration time= 0.0071985721588134766
max ddh =  nan
step = 256  avg d_h= nan
Number of iterations = 1 tolerance = nan  iterat

KeyboardInterrupt: 

In [6]:
subplot(2,2,1)
imshow_grid_at_node(mg, h, var_name='Head ($m$)', cmap='viridis',vmin=300,vmax=400)
        #plot(mg.at_node['hydraulic__head'][mg.core_nodes])
        #ylim([300,400])
subplot(2,2,2)
plot_links(mg, 'conduit__discharge', magnitude=True, var_name='Discharge ($m^3/s$)', vmin=0., vmax=1, autoscale=False, pixels_per_node=500)
#plot(mg.at_link['conduit__discharge'][mg.active_links])
#ylabel('')
#ylim([0,1.5])
subplot(2,2,3)
plot_links(mg, 'hydraulic__diameter', var_name='Hydraulic diameter ($m$)', autoscale=False, vmin=0, vmax=1.25, pixels_per_node=500)
subplot(2,2,4)#, position=[0.2,0.2,0.9,0.9])
plot(mg.at_link['hydraulic__diameter'][mg.active_links])
xlabel('Link number')
ylabel('Hydraulic diameter ($m$)')
ylim(0,1.25)
#ylim([0.5,1.5])
image_name = 'heads_and_discharge'+str(step).zfill(6)+'.png'
#tight_layout()
subplots_adjust(wspace=0.3)
#savefig(image_name)

image data type = [[[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]]

 [[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]]

 [[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]]]
image data type = [[[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]]

 [[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]]

 [[ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84 255]
  [ 68   1  84

In [5]:
figure()
imshow_grid_at_node(mg, h)
title('head')

figure()
plot_links(mg, 'conduit__discharge')#,logvalues=True)
title('discharge')


figure()
plot_links(mg, 'hydraulic__diameter')#,logvalues=True)
title('d_H')


image data type = <PIL.Image.Image image mode=RGBA size=0x0 at 0x7F558C2C5B80>


TypeError: Image data of dtype object cannot be converted to float

In [7]:
mg.at_link['conduit__discharge'].dtype

dtype('float64')

In [5]:
figure()
plot(mg.at_link['conduit__discharge'][mg.active_links])

In [16]:
mc.run_one_step()
pfn.run_one_step()

#mc = MeltCreep(mg, dt=5000.)


max ddh =  4.545746571648965e-05
Number of iterations = 1 tolerance = 6.646520870778883e-05
Number of iterations = 2 tolerance = 8.926521188890035e-09


In [17]:
dhdx = mg.add_zeros('link', 'hydraulic_head_gradient')
net_node_flux = mg.add_ones('node', 'net_node_flux')


In [1]:


mg.calc_net_flux_at_node(Q, out=net_node_flux)
net_Q = mg.calc_net_flux_at_node(Q)/mg.dx - mg.at_node['input__discharge'] 
figure()
imshow_grid_at_node(mg, net_Q)

NameError: name 'mg' is not defined

In [39]:
max(mg.calc_net_flux_at_node(Q)/mg.dx - mg.at_node['input__discharge'])

6.0770875975890707e-09

In [24]:
from scipy.optimize import fsolve
def network_residuals(heads):
    h[mg.core_nodes] = heads
    mg.calc_grad_at_link(h, out=dhdx)
    Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    return mg.calc_net_flux_at_node(Q, out=net_node_flux)[mg.core_nodes]/mg.dx - mg.at_node['input__discharge'][mg.core_nodes]
#This function doesn't account for inflow at nodes


In [25]:
mc.run_one_step()
mg.calc_grad_at_link(h, out=dhdx)
mg.at_node['hydraulic__head'][mg.core_nodes] = fsolve(network_residuals, h[mg.core_nodes])
mg.at_link['conduit__discharge'][mg.core_nodes] = np.sign(dhdx[mg.core_nodes])*np.sqrt(np.fabs(dhdx[mg.core_nodes]))

print "mean h =", mean(mg.at_node['hydraulic__head'][mg.core_nodes] )
print "mean Q =", mean(Q)

mean ddh =  -5.66192997218e-18
mean h = 166.222550077
mean Q = 0.0865514465643


In [22]:
mg.dx

100.0

In [26]:
figure()
imshow_grid_at_node(mg, h)

In [24]:
figure()
plot_links(mg, 'hydraulic__diameter', autoscale=False, vmin=0, vmax=1.)
print mean(mg.at_link['hydraulic__diameter'])

3.50074727364e+69


In [56]:
figure()
plot_links(mg, 'conduit__discharge', autoscale=True, magnitude=True)
#print mean(mg.at_link['hydraulic__diameter'])

In [10]:
mc = MeltCreep(mg, dt=5000.)
#mean(mc.Q)

In [17]:
plot(mg.at_link['hydraulic__diameter'][200:300])


Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902


Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.6482677381

In [19]:
#figure()

pfn.run_one_step()
mc.run_one_step()

Number of iterations = 1 tolerance = 0.0031818002267
Number of iterations = 2 tolerance = 1.26436621244e-05
mean ddh =  -6.86035808489e-06


In [41]:
#for m in mc.melt:
#    print m
print max(mc.melt[~np.isnan(mc.melt)])
print max(mc.creep)
#print mc.d_h.mean()
#print mg.at_link['hydraulic__diameter']

3.35041761374e-06
2.12710591355e-06


In [15]:
#mc.run_one_step()

mc._grid.at_link['conduit__discharge']

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [33]:
mc.run_one_step()
pfn.run_one_step_fsolve()

max ddh =  7.24228976607e-06


max ddh =  7.26879292542e-06
